***提取出csv训练数据并作成字典方便标注使用***

In [1]:
#出院小结
def get_discharge():
    discharge_summary={
        "性别":"无","年龄":"无","医院名称":"无","入院日期":"无","出院日期":"无",
        "组织机构代码":"无","医疗机构类型":"无","住院天数":"无"
        }
    return discharge_summary

#购药发票
def get_medicines():
    medicines_invoice={
        "票据代码":"无","票据号码":"无","校验码":"无",
        "开票日期":"无","收款人":"无","复核人":"无","价税合计（大写）":"无","（小写）":"无"
        }
    return medicines_invoice

#门诊发票
def get_outpatient():
    outpatient_invoice={
        "性别":"无","票据代码":"无","票据号码":"无","校验码":"无","开票日期":"无","收款单位":"无",
        "收款人":"无","复核人":"无","医疗机构类型":"无","医保类型":"无","合计金额（大写）":"无","（小写）":"无",
        "诊查费":"无","检查费":"无","化验费":"无","治疗费":"无","手术费":"无","卫生材料费":"无",
        "西药费":"无","中药饮片":"无","中成药费":"无","一般诊疗费":"无","床位费":"无","护理费":"无",
        "挂号费":"无","其他收费项目":"无","医保统筹基金支付":"无","统筹支付":"无","个人现金支付":"无","个人账户支付":"无",
        "个人自付":"无","其他支付":"无","自付一":"无","自付二":"无"
    }
    return outpatient_invoice

#住院发票
def get_hospitalization():
    hospitalization_invoice={
        "性别":"无","住院天数":"无","票据代码":"无","票据号码":"无","校验码":"无","开票日期":"无","收款单位":"无","入院日期":"无","出院日期":"无",
        "收款人":"无","复核人":"无","医疗机构类型":"无","医保类型":"无","合计金额（大写）":"无","（小写）":"无",
        "诊查费":"无","检查费":"无","化验费":"无","治疗费":"无","手术费":"无","卫生材料费":"无",
        "西药费":"无","中药饮片":"无","中成药费":"无","一般诊疗费":"无","床位费":"无","护理费":"无",
        "挂号费":"无","其他收费项目":"无","医保统筹基金支付":"无","统筹支付":"无","个人现金支付":"无","个人账户支付":"无",
        "个人自付":"无","其他支付":"无","自付一":"无","自付二":"无"
    }
    return hospitalization_invoice

In [2]:
from datasets import load_dataset
traindata=load_dataset("csv",data_files="datasets\大赛1000训练用数据集.csv")
dataset=traindata["train"]

Using custom data configuration default-d3fba8b5dbc43d09
Reusing dataset csv (D:\hugging-face\datasets\csv\default-d3fba8b5dbc43d09\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/1 [00:00<?, ?it/s]

***得到一些路径变量等信息***

In [3]:
import ast
import os

def get_imlist(path):
        return [os.path.join(path,f) for f in os.listdir(path)]

path="train_out_txt"
pathlist=get_imlist(path)

def get_name_list(path):
    with open(path,encoding="utf-8") as f:
        lists=ast.literal_eval(f.readline())
        #获取到文本列表
        return os.path.basename(path),lists

def get_str(listdata):
    res=""
    for item in listdata:
        res=res+str(item)+","
    return res

***获取需要标注的数据***

In [4]:
waiting={}
for item in pathlist:
    basename,lists=get_name_list(item)
    txtstr=get_str(lists)
    waiting[basename]=txtstr

In [5]:
def isChinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

def Reverse(lst):
    return [ele for ele in reversed(lst)]

def cut_words(words):
    res=[]
    number=""
    bd=["年","月","日","时","分","秒"]
    
    for word in words:
        if (isChinese(word) and (word not in bd)) or (word=="：" or word==":" or word=="￥" or word==","):
            if number!="" :
                res.append(number)
            number=""
            res.append(word)
        else:
            number+=word
    
    if number!="":
        res.append(number)
    
    result=res.copy()

    for item in res:
        temp=item
        newa=[]
        if item.count(".")>1:
            a=item.split(".")
            for i in range(len(a)):
                value=""
                if i==0:
                    value=a[i]+"."+a[i+1][:2]
                    newa.append(value)

                elif i!=len(a)-1:
                    value=a[i][2:]+"."+a[i+1][:2]
                    newa.append(value)
            
            num=[0,1,2,3,4,5,6,7,8,9]
            mark=0
            for item in str(a[-1]):
                if item not in num:
                    mark=1

            if len(a[-1])>2 and mark==0:
                a[-1]=int(a[-1])/100
                newa.append(str(a[-1]))

        if len(newa)>1:
            newa=Reverse(newa)

            for i in range(len(result)):
                if result[i]==temp:
                    del result[i]
                    for j in range(len(newa)):
                        result.insert(i,newa[j])

    return result

In [6]:
res=[]
for filename,sentence in waiting.items():
    if filename=="d49df836d17ce8c3651594b85c468a02.txt":
        res=cut_words(sentence)

data=[]
for item in dataset:
    if item["图名"]=="d49df836d17ce8c3651594b85c468a02":
        data.append(item["正确值"])
value=[]
for item in data:
    item=cut_words(item)
    value.append(item)

***对单个字符串进行标注***

In [7]:
def mark(cuted_txt,dataset,token2label,label2id):
    l1=len(cuted_txt)
    res=[]
    for i in range(l1):
        res.append("O")
    
    for key,value in dataset.items():
        value=cut_words(value)
        
        if value[0] == "无":
            continue
        
        l2=len(value)
        flag=0
        for i in range(l1):
            if flag==1:
                break
            
            if cuted_txt[i] == value[0] and res[i]=="O":
                if l2>1:
                    if i!=(l1-1) and (i+l2-1)<l1:
                        if cuted_txt[i+1] == value[1] and cuted_txt[i+l2-1] == value[-1]:
                            res[i]=token2label[key+"B"]
                            for j in range(i+1,i+l2):
                                res[j]=token2label[key+"I"]
                            flag=1
                else:
                    res[i]=token2label[key+"B"]
                    flag=1
        
        if flag==0: 
            for i in range(l1):
                if key=="票据代码" or key=="票据号码" or key[-1]=="费":
                    if res[i]=="O":
                        if (value[0] in cuted_txt[i] or cuted_txt[i] in value[0]):
                            res[i]=token2label[key+"B"]

                if cuted_txt[i]==value[0] and res[i]=="O":
                    if key=="入院日期":
                        if i!=(l1-1) and (i+l2-1)<l1:
                            if cuted_txt[i+l2-1]==value[-1] and cuted_txt[i+l2-2]==value[-2]:
                                res[i]=token2label[key+"B"]
                                for j in range(i+1,i+l2):
                                    res[j]=token2label[key+"I"]
                                flag=1
                    elif key=="出院日期":
                        if i!=(l1-1) and (i+l2-1)<l1:
                            if cuted_txt[i+l2-1]==value[-1] and cuted_txt[i+l2-2]==value[-2]:
                                res[i]=token2label[key+"B"]
                                for j in range(i+1,i+l2):
                                    res[j]=token2label[key+"I"]
                                flag=1
                    
                    
                    if key!="出院日期" and key!="入院日期" and key!="票据号码" and key!="票据代码":
                        if i!=(l1-1) and (i+l2-1)<l1:
                            if cuted_txt[i+1]==value[1]:
                                for j in range(i+1,i+1+l2):
                                    if j>l1-1:
                                        res[i]=token2label[key+"B"]
                                        for l in range(i+1,l1-1):
                                            res[l]=token2label[key+"I"]
                                        flag=1
                                        break
                                    if cuted_txt[j]==value[-1] :
                                        res[i]=token2label[key+"B"]
                                        for l in range(i+1,j):
                                            res[l]=token2label[key+"I"]
                                        flag=1
    
    ner_tags=[]
    for item in res:
        ner_tags.append(int(label2id[item]))


    label2token={}
    for key,value in token2label.items():
        label2token[value]=key

    for item in res:
        if item!="O":
            print(label2token[item],end=" ")

    print("\n")
    return res,ner_tags

***出院小结对应的标注标签***

In [8]:
discharge_label=["O","B-G","I-G","B-A","I-A","B-N","I-N","B-C","I-C","B-L","I-L","B-D","I-D","B-IN","I-IN","B-OUT","I-OUT"]
#出院小结标注对应字典
token2label={"无":"O","性别B":"B-G","性别I":"I-G","年龄B":"B-A","年龄I":"I-A","医院名称B":"B-N","医院名称I":"I-N","组织机构代码B":"B-C","组织机构代码I":"I-C","医疗机构类型B":"B-L","医疗机构类型I":"I-L","住院天数B":"B-D","住院天数I":"I-D","入院日期B":"B-IN","入院日期I":"I-IN","出院日期B":"B-OUT","出院日期I":"I-OUT"}

id2label = {str(i): label for i, label in enumerate(discharge_label)}
label2id = {v: k for k, v in id2label.items()}
'''G代表性别、A代表年龄、N代表医院名称、C代表组织机构代码、L代表医疗机构类型、
IN代表入院日期、OUT代表出院日期、D代表住院天数、'''
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-G', '2': 'I-G', '3': 'B-A', '4': 'I-A', '5': 'B-N', '6': 'I-N', '7': 'B-C', '8': 'I-C', '9': 'B-L', '10': 'I-L', '11': 'B-D', '12': 'I-D', '13': 'B-IN', '14': 'I-IN', '15': 'B-OUT', '16': 'I-OUT'}
{'O': '0', 'B-G': '1', 'I-G': '2', 'B-A': '3', 'I-A': '4', 'B-N': '5', 'I-N': '6', 'B-C': '7', 'I-C': '8', 'B-L': '9', 'I-L': '10', 'B-D': '11', 'I-D': '12', 'B-IN': '13', 'I-IN': '14', 'B-OUT': '15', 'I-OUT': '16'}


***对所有出院小结进行标注***

In [9]:
import json
marked=[]
datasets={}
for item in dataset:
    if item['图名'] not in marked:
        marked.append(item['图名'])
        ctype=item['材料类型']
        if ctype=="出院小结":
            datasets[item['图名']]=get_discharge()


for item in dataset:
    ctype=item['材料类型']
    if ctype=="出院小结":
        datasets[item['图名']][item["属性名"]]=item['正确值']



discharge_res={"id":[],"tokens":[],"ner_tags":[]}
id=0
for filename,sentence in waiting.items():
    type_true=0
    tokens=cut_words(sentence)
    name=filename.split(".")[0]
    for dataname,feature in datasets.items():
        if name in dataname and name==dataname.split(".")[0]:
            print(dataname,end="\t")
            res,ner_tags=mark(tokens,feature,token2label,label2id)
            discharge_res["id"].append(id)
            discharge_res["tokens"].append(tokens)
            discharge_res["ner_tags"].append(ner_tags)
            type_true=1
            break
    if type_true==0:
        continue
    id+=1


with open("NER_dataset/discharge_res.json", 'w', encoding="utf-8") as write_f:
	json.dump(discharge_res, write_f, indent=4, ensure_ascii=False)

001bfce2288c0cbe0a2811	入院日期B 出院日期B 住院天数B 

00630d24effe4917ac4551cec2d203bf_14	性别B 年龄B 年龄I 住院天数B 住院天数I 入院日期B 

012ad7ed5f734d9386108a0e9b04f0ec_17	医院名称B 医院名称I 医院名称I 医院名称I 医院名称I 入院日期B 出院日期B 

020170518372271232793	医院名称B 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 年龄B 年龄I 性别B 住院天数B 

02894c97699c469da59853dab2c6fa1e_50	性别B 住院天数B 住院天数I 入院日期B 出院日期B 

02894c97699c469da59853dab2c6fa1e_9	性别B 住院天数B 住院天数I 入院日期B 出院日期B 

06065805fcfa415a857bd14999062c79_19	医院名称B 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 性别B 年龄B 年龄I 入院日期B 出院日期B 

073aa20c7eff4c1e9deb6e6b2cbc0a45_12	性别B 入院日期B 

091c681695ac40d796deb0d07d4e8dbf_36	医院名称B 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 性别B 年龄B 年龄I 入院日期B 出院日期B 

0adf0dfad93a4827ad8fb3bfd91eb4f0_11	医院名称B 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 性别B 年龄B 年龄I 入院日期B 

0ae166a8804441acbecfd38e6c6411de_11	医院名称B 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 医院名称I 

***购药发票对应的标注标签***

In [10]:
medicines_label = ["O", "B-PD", "I-PD",  
                   "B-PH", "I-PH",
                   "B-JY", "I-JY", 
                   "B-PR", "I-PR", 
                   "B-SR", "I-SR", 
                   "B-FR", "I-FR",                    
                   "B-JD","I-JD",
                    "B-JX", "I-JX", ]
# 购药发票标注对应字典
token2label = {"无": "O", "票据代码B": "B-PD", "票据代码I": "I-PD",
                "票据号码B": "B-PH", "票据号码I": "I-PH", 
               "校验码B": "B-JY", "校验码I": "I-JY", 
               "开票日期B": "B-PR", "开票日期I": "I-PR",
                "收款人B": "B-SR", "收款人I": "I-SR", 
                "复核人B": "B-FR", "复核人I": "I-FR",
                "价税合计（大写）B":"B-JD","价税合计（大写）I":"I-JD",
                "（小写）B":"B-JX","（小写）I":"I-JX"
               }


id2label = {str(i): label for i, label in enumerate(medicines_label)}
label2id = {v: k for k, v in id2label.items()}
''' 票据代码-PD、票据号码-PH、校验码-JY、开票日期-PR、收款人-SR、复核人-FR、
    价税合计（大写）B-JD、（小写）-JX、'''
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-PD', '2': 'I-PD', '3': 'B-PH', '4': 'I-PH', '5': 'B-JY', '6': 'I-JY', '7': 'B-PR', '8': 'I-PR', '9': 'B-SR', '10': 'I-SR', '11': 'B-FR', '12': 'I-FR', '13': 'B-JD', '14': 'I-JD', '15': 'B-JX', '16': 'I-JX'}
{'O': '0', 'B-PD': '1', 'I-PD': '2', 'B-PH': '3', 'I-PH': '4', 'B-JY': '5', 'I-JY': '6', 'B-PR': '7', 'I-PR': '8', 'B-SR': '9', 'I-SR': '10', 'B-FR': '11', 'I-FR': '12', 'B-JD': '13', 'I-JD': '14', 'B-JX': '15', 'I-JX': '16'}


***对所有购药发票进行标注***

In [11]:
import json
marked=[]
datasets={}
for item in dataset:
    if item['图名'] not in marked:
        marked.append(item['图名'])
        ctype=item['材料类型']
        if ctype=="购药发票":
            datasets[item['图名']]=get_medicines()


for item in dataset:
    ctype=item['材料类型']
    if ctype=="购药发票":
        datasets[item['图名']][item["属性名"]]=item['正确值']



medicines_res={"id":[],"tokens":[],"ner_tags":[]}
id=0
for filename,sentence in waiting.items():
    type_true=0
    tokens=cut_words(sentence)
    name=filename.split(".")[0]
    for dataname,feature in datasets.items():
        if name in dataname and name==dataname.split(".")[0]:
            print(dataname,end="\t")
            res,ner_tags=mark(tokens,feature,token2label,label2id)
            medicines_res["id"].append(id)
            medicines_res["tokens"].append(tokens)
            medicines_res["ner_tags"].append(ner_tags)
            type_true=1
    if type_true==0:
        continue
    id+=1



with open("NER_dataset/medicines_res.json", 'w', encoding="utf-8") as write_f:
	json.dump(medicines_res, write_f, indent=4, ensure_ascii=False)

0025b5f3b7b4f5b4bad3219e08f85d84	票据代码B 票据号码B 开票日期B （小写）B 

027A3313DDD6453DA988D59A84CD5D8D	票据代码B 票据号码B 开票日期B 价税合计（大写）B 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I （小写）B 

049da812b559c30f75eff5ea81b5c60d	票据代码B 票据号码B 开票日期B 价税合计（大写）B 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 

067095ae2ec9e8ae542a89509acf3c60	开票日期B 票据代码B 票据代码B 价税合计（大写）B 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 收款人B 收款人I 收款人I 复核人B 复核人I 复核人I 

070d57cf-e29e-4e56-947e-7269754ab1f2	票据代码B 票据号码B 开票日期B 校验码B 收款人B 价税合计（大写）B 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I （小写）B 

09e2ffc1adb1d3d2493f977056cb64e6	票据代码B 票据号码B 开票日期B （小写）B 收款人B 收款人I 收款人I 复核人B 复核人I 复核人I 

0bdc48e67262686922c228dff80ddcb7	票据代码B 票据号码B 开票日期B 校验码B 价税合计（大写）B 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I （小写）B 复核人B 复核人I 复核人I 收款人B 收款人I 收款人I 

1-14091Q52005915	票据代码B 票据号码B 开票日期B 价税合计（大写）B 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（大写）I 价税合计（

***住院发票和门诊发票合在一起标注***

In [12]:
labels_name = ["O", "B-G", "I-G", "B-D", "I-D", "B-PD", "I-PD", "B-PH", "I-PH","B-IN","I-IN","B-OUT","I-OUT",
                   "B-JY", "I-JY", "B-PR", "I-PR", "B-SD", "I-SD", "B-SR", "I-SR", "B-FR", "I-FR", "B-L", "I-L", "B-YL",
                   "I-YL", "B-ZC", "I-ZC", "B-JC", "I-JC", "B-HY", "I-HY", "B-ZL", "I-ZL", "B-S", "I-S", "B-WC", "I-WC",
                   "B-WY", "I-WY", "B-ZY", "I-ZY", "B-ZCY", "I-ZCY", "B-YZL", "I-YZL", "B-CW", "I-CW", "B-HL", "I-HL",
                   "B-GH", "I-GH", "B-ELE", "I-ELE", "B-HJD", "I-HJD", "B-HJX", "I-HJX", "B-YTC", "I-YTC", "B-TC",
                   "I-TC", "B-PX", "I-PX", "B-PP", "I-PP", "B-PO", "I-PO", "B-ELP", "I-ELP", "B-POO", "I-POO", "B-PW",
                   "I-PW"]
# 住院发票标注对应字典
token2label = {"无": "O", "性别B": "B-G", "性别I": "I-G", "入院日期B":"B-IN","入院日期I":"I-IN","出院日期B":"B-OUT","出院日期I":"I-OUT",
               "住院天数B": "B-D", "住院天数I": "I-D", "票据代码B": "B-PD", "票据代码I": "I-PD",
               "票据号码B": "B-PH", "票据号码I": "I-PH", "校验码B": "B-JY", "校验码I": "I-JY", "开票日期B": "B-PR", "开票日期I": "I-PR",
               "收款单位B": "B-SD", "收款单位I": "I-SD", "收款人B": "B-SR", "收款人I": "I-SR", "复核人B": "B-FR", "复核人I": "I-FR",
               "医疗机构类型B": "B-L", "医疗机构类型I": "I-L", "医保类型B": "B-YL", "医保类型I": "I-YL", "诊查费B": "B-ZC", "诊查费I": "I-ZC",
               "检查费B": "B-JC", "检查费I": "I-JC", "化验费B": "B-HY", "化验费I": "I-HY", "治疗费B": "B-ZL", "治疗费I": "I-ZL",
               "手术费B": "B-S", "手术费I": "I-S", "卫生材料费B": "B-WC", "卫生材料费I": "I-WC", "西药费B": "B-WY", "西药费I": "I-WY",
               "中药饮片B": "B-ZY", "中药饮片I": "I-ZY", "中成药费B": "B-ZCY", "中成药费I": "I-ZCY", "一般诊疗费B": "B-YZL",
               "一般诊疗费I": "I-YZL",
               "床位费B": "B-CW", "床位费I": "I-CW", "护理费B": "B-HL", "护理费I": "I-HL", "挂号费B": "B-GH", "挂号费I": "I-GH",
               "其他收费项目B": "B-ELE", "其他收费项目I": "I-ELE", "合计金额（大写）B": "B-HJD", "合计金额（大写）I": "I-HJD",
               "（小写）B": "B-HJX", "（小写）I": "I-HJX", "医保统筹基金支付B": "B-YTC", "医保统筹基金支付I": "I-YTC", "统筹支付B": "B-TC",
               "统筹支付I": "I-TC", "个人现金支付B": "B-PX", "个人现金支付I": "I-PX", "个人账户支付B": "B-PP", "个人账户支付I": "I-PP",
               "个人自付B": "B-PO", "个人自付I": "I-PO", "其他支付B": "B-ELP", "其他支付I": "I-ELP", "自付一B": "B-POO", "自付一I": "I-POO",
               "自付二B": "B-PW", "自付二I": "I-PW"
               }

id2label = {str(i): label for i, label in enumerate(labels_name)}
label2id = {v: k for k, v in id2label.items()}
'''客户信息字段（属性名共 4个）：性别-G、入院日期-IN、出院日期-OUT、住院天数-D
   发票信息字段（属性名共 7个）：票据代码-PD、票据号码-PH、校验码-JY、开票日期-PR、收款单位-SD、收款人-SR、复核人-FR
   医保信息字段（属性名共 2个）：医疗机构类型-L、医保类型-YL
   项目信息字段（属性名共 14个）：诊查费-ZC、检查费-JC、化验费-HY、治疗费-ZL、手术费-S、卫生材料费-WC、西药费-WY、中药饮片-ZY、中成药费-ZCY、一般诊疗费-YZL、床位费-CW、护理费-HL、挂号费-GH、其他收费项目-ELE
   支付信息字段（属性名共 10个）：合计金额（大写）-HJD、（小写）-HJX、医保统筹基金支付-YTC、统筹支付-TC、个人现金支付-PX、个人账户支付-PP、个人自付-PO、其他支付-ELP、自付一-POO、自付二-PW'''

'客户信息字段（属性名共 4个）：性别-G、入院日期-IN、出院日期-OUT、住院天数-D\n   发票信息字段（属性名共 7个）：票据代码-PD、票据号码-PH、校验码-JY、开票日期-PR、收款单位-SD、收款人-SR、复核人-FR\n   医保信息字段（属性名共 2个）：医疗机构类型-L、医保类型-YL\n   项目信息字段（属性名共 14个）：诊查费-ZC、检查费-JC、化验费-HY、治疗费-ZL、手术费-S、卫生材料费-WC、西药费-WY、中药饮片-ZY、中成药费-ZCY、一般诊疗费-YZL、床位费-CW、护理费-HL、挂号费-GH、其他收费项目-ELE\n   支付信息字段（属性名共 10个）：合计金额（大写）-HJD、（小写）-HJX、医保统筹基金支付-YTC、统筹支付-TC、个人现金支付-PX、个人账户支付-PP、个人自付-PO、其他支付-ELP、自付一-POO、自付二-PW'

In [13]:
import json
marked=[]
datasets={}
for item in dataset:
    if item['图名'] not in marked:
        marked.append(item['图名'])
        ctype=item['材料类型']
        if ctype=="住院发票" or ctype=="门诊发票":
            datasets[item['图名']]=get_hospitalization()


for item in dataset:
    ctype=item['材料类型']
    if ctype=="住院发票" or ctype=="门诊发票":
        datasets[item['图名']][item["属性名"]]=item['正确值']



fp_res={"id":[],"tokens":[],"ner_tags":[]}
id=0
for filename,sentence in waiting.items():
    type_true=0
    tokens=cut_words(sentence)
    name=filename.split(".")[0]
    for dataname,feature in datasets.items():
        if name in dataname and name==dataname.split(".")[0]:
            print(dataname,end="\t")
            res,ner_tags=mark(tokens,feature,token2label,label2id)
            fp_res["id"].append(id)
            fp_res["tokens"].append(tokens)
            fp_res["ner_tags"].append(ner_tags)
            type_true=1
    if type_true==0:
        continue
    id+=1



with open("NER_dataset/fp_res.json", 'w', encoding="utf-8") as write_f:
	json.dump(fp_res, write_f, indent=4, ensure_ascii=False)

00dca88e-d75b-4882-a1d8-ffc4ff29f2fc.jpg	医保类型B 医保类型I 票据号码B 护理费B 西药费B 化验费B （小写）B 合计金额（大写）B 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 

0113007771ed86a1fdaca3e316c4ddd6	票据号码B 医保类型B 医保类型I 医保类型I 医保类型I 性别B 治疗费B 护理费B 化验费B （小写）B 检查费B 个人账户支付B 检查费B 检查费B 

0114fac7a3b3cbb9a2e94c0e65e7cad7.jpeg	票据号码B 开票日期B 西药费B 检查费B 手术费B 化验费B 中药饮片B 床位费B 治疗费B 一般诊疗费B 中成药费B 性别B 个人账户支付B 医保统筹基金支付B 收款人B 收款人I 收款人I 复核人B 复核人I 复核人I 

01505a3a43_6	票据代码B 票据号码B 检查费B 校验码B 开票日期B 护理费B 化验费B 手术费B 化验费B 治疗费B 检查费B 化验费B 护理费B 化验费B 床位费B 治疗费B 治疗费B 西药费B 检查费B 检查费B 手术费B 诊查费B 化验费B 化验费B 合计金额（大写）B 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 合计金额（大写）I 检查费B 检查费B 手术费B 个人账户支付B 检查费B 医疗机构类型B 医疗机构类型I 医疗机构类型I 医疗机构类型I 医保类型B 医保类型I 医保类型I 医保类型I 医保类型I 化验费B 性别B 治疗费B 个人自付B 检查费B 检查费B 收款人B 收款人I 复核人B 复核人I 

015a905818_46.jpg	票据号码B 票据号码B 西药费B 化验费B 手术费B 检查费B 护理费B 票据代码B 

01922ebc7a_7	票据代码B 票据号码B 校验码B 开票日期B 中成药费B 床位费B 化验费B 西药费B 西药费B 手术费B 手术费B 治疗费B 护理费B